# Cleaning datas 

We give a preliminary clean of the datas, before cleaning and analyzing them further for specific models in different notebooks. We obtained historical weather datas for Montreal from OpenWeatherMap. The datas are from January 1st, 1979 to July 31st, 2020. 

The collected features are: 

- <code> city_name </code> City name
- <code> lat </code> Geographical coordinates of the location (latitude)
- <code> lon </code> Geographical coordinates of the location (longitude)
- <code> main </code>
    - <code> main.temp </code> Temperature
    - <code> main.feels_like </code> This temperature parameter accounts for the human perception of weather
    - <code> main.pressure </code> Atmospheric pressure (on the sea level), hPa
    - <code> main.humidity </code> Humidity, %
    - <code> main.temp_min </code> Minimum temperature at the moment. This is deviation from temperature that is possible for large cities and megalopolises geographically expanded (use these parameter optionally).
    - <code> main.temp_max </code> Maximum temperature at the moment. This is deviation from temperature that is possible for large cities and megalopolises geographically expanded (use these parameter optionally).
- <code> wind </code>
    - <code> wind.speed </code> Wind speed. Unit Default: meter/sec
    - <code> wind.deg </code> Wind direction, degrees (meteorological)
- <code> clouds </code>
    - <code> clouds.all </code> Cloudiness, %
- <code> rain </code>
    - <code> rain.1h </code> Rain volume for the last hour, mm
    - <code> rain.3h </code> Rain volume for the last 3 hours, mm
- <code> snow </code>
    - <code> snow.1h </code> Snow volume for the last hour, mm (in liquid state)
    - <code> snow.3h </code> Snow volume for the last 3 hours, mm (in liquid state)
- <code> weather </code> 
    - <code> weather.id </code> Weather condition id
    - <code> weather.main </code> Group of weather parameters (Rain, Snow, Extreme etc.)
    - <code> weather.description </code> Weather condition within the group
    - <code> weather.icon </code> Weather icon id
- <code> dt </code> Time of data calculation, unix, UTC
- <code> dt_iso </code> Date and time in UTC format
- <code> timezone </code> Shift in seconds from UTC

The explanation for the weather condition id and icon id can be found here: https://openweathermap.org/weather-conditions

We import the useful libraries. 

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

We read the csv file.

In [2]:
df_data = pd.read_csv('weather_data_montreal.csv')

print('Data Shape = {}'.format(df_data.shape))
print(df_data.columns)

Data Shape = (373025, 25)
Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'sea_level',
       'grnd_level', 'humidity', 'wind_speed', 'wind_deg', 'rain_1h',
       'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'weather_icon'],
      dtype='object')


In [3]:
pd.set_option('display.max_columns', 999)
print(df_data.info())
df_data.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373025 entries, 0 to 373024
Data columns (total 25 columns):
dt                     373025 non-null int64
dt_iso                 373025 non-null object
timezone               373025 non-null int64
city_name              373025 non-null object
lat                    373025 non-null float64
lon                    373025 non-null float64
temp                   373025 non-null float64
feels_like             373025 non-null float64
temp_min               373025 non-null float64
temp_max               373025 non-null float64
pressure               373025 non-null int64
sea_level              0 non-null float64
grnd_level             0 non-null float64
humidity               373025 non-null int64
wind_speed             373025 non-null float64
wind_deg               373025 non-null int64
rain_1h                36787 non-null float64
rain_3h                1622 non-null float64
snow_1h                11761 non-null float64
snow_3h               

,dt,dt_iso,timezone,city_name,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
100483,629665200,1989-12-14 19:00:00 +0000 UTC,-18000,Montreal,45.501689,-73.567256,254.98,246.18,254.504,255.150,1017,NaN,NaN,49,7.2,250,NaN,NaN,NaN,NaN,20,801,Clouds,few clouds,02d
324435,1421931600,2015-01-22 13:00:00 +0000 UTC,-18000,Montreal,45.501689,-73.567256,254.58,249.15,252.347,256.150,1025,NaN,NaN,84,2.6,10,NaN,NaN,NaN,NaN,1,800,Clear,sky is clear,01d
122861,706017600,1992-05-16 12:00:00 +0000 UTC,-14400,Montreal,45.501689,-73.567256,286.12,283.83,286.037,286.201,1029,NaN,NaN,93,4.1,330,0.30,NaN,NaN,NaN,90,520,Rain,light intensity shower rain,09d
148331,794779200,1995-03-09 20:00:00 +0000 UTC,-18000,Montreal,45.501689,-73.567256,266.75,259.39,266.150,267.307,1020,NaN,NaN,73,6.1,270,NaN,NaN,NaN,NaN,90,600,Snow,light snow,13d
148281,794617200,1995-03-07 23:00:00 +0000 UTC,-18000,Montreal,45.501689,-73.567256,279.46,269.75,278.835,280.150,1011,NaN,NaN,81,11.8,180,0.13,NaN,NaN,NaN,90,520,Rain,light intensity shower rain,09n


We change the 'dt_iso' into a datetime format. Then we remove 'dt' and 'timezone', as we will keep the dates and times with respect to UTC. 

In [4]:
df_data['dt_iso'] = df_data['dt_iso'].map(lambda x: x.replace('+0000 UTC', ''))
df_data['dt_iso'] = pd.to_datetime(df_data['dt_iso'], format='%Y-%m-%d %H:%M:%S.%f')
print(df_data.head())
print(df_data.info())

          dt              dt_iso  timezone city_name        lat        lon  \
0  283996800 1979-01-01 00:00:00    -18000  Montreal  45.501689 -73.567256   
1  284000400 1979-01-01 01:00:00    -18000  Montreal  45.501689 -73.567256   
2  284004000 1979-01-01 02:00:00    -18000  Montreal  45.501689 -73.567256   
3  284007600 1979-01-01 03:00:00    -18000  Montreal  45.501689 -73.567256   
4  284011200 1979-01-01 04:00:00    -18000  Montreal  45.501689 -73.567256   

     temp  feels_like  temp_min  temp_max  pressure  sea_level  grnd_level  \
0  275.12      269.76   274.736   275.443      1025        NaN         NaN   
1  275.08      271.53   274.774   275.305      1023        NaN         NaN   
2  275.06      271.16   274.762   275.217      1022        NaN         NaN   
3  275.97      267.30   275.150   276.952      1021        NaN         NaN   
4  276.32      267.88   276.150   276.862      1019        NaN         NaN   

   humidity  wind_speed  wind_deg  rain_1h  rain_3h  snow_1h  

In [5]:
df_data = df_data.drop(columns = ['dt', 'timezone'])

Here we have 373025 rows. However, there should only be 364512 hours between January 1, 1979 and July 31, 2020. This means that there could be duplicates. We search for them. 

In [6]:
duplicate = df_data[df_data.duplicated(subset =['dt_iso'], keep = False)] 
  
print("Duplicate Rows :") 
  
# Print the resultant Dataframe 
duplicate 

Duplicate Rows :


,dt_iso,city_name,lat,lon,temp,feels_like,temp_min,temp_max,pressure,sea_level,grnd_level,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
4,1979-01-01 04:00:00,Montreal,45.501689,-73.567256,276.32,267.88,276.150,276.862,1019,NaN,NaN,93,9.7,160,0.50,NaN,NaN,NaN,90,500,Rain,light rain,10n
5,1979-01-01 04:00:00,Montreal,45.501689,-73.567256,276.32,267.88,276.150,276.862,1019,NaN,NaN,93,9.7,160,0.50,NaN,NaN,NaN,90,741,Fog,fog,50n
6,1979-01-01 05:00:00,Montreal,45.501689,-73.567256,276.27,266.53,275.150,277.150,1017,NaN,NaN,86,11.3,170,0.50,NaN,NaN,NaN,90,500,Rain,light rain,10n
7,1979-01-01 05:00:00,Montreal,45.501689,-73.567256,276.27,266.53,275.150,277.150,1017,NaN,NaN,86,11.3,170,0.50,NaN,NaN,NaN,90,741,Fog,fog,50n
8,1979-01-01 06:00:00,Montreal,45.501689,-73.567256,276.59,267.43,276.118,277.150,1015,NaN,NaN,93,10.8,170,0.50,NaN,NaN,NaN,90,500,Rain,light rain,10n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372919,2020-07-27 16:00:00,Montreal,45.501689,-73.567256,298.47,300.47,296.615,300.930,991,NaN,NaN,94,5.7,240,0.40,NaN,NaN,NaN,90,701,Mist,mist,50d
372923,2020-07-27 20:00:00,Montreal,45.501689,-73.567256,300.47,304.83,298.710,303.150,981,NaN,NaN,94,4.1,270,1.85,NaN,NaN,NaN,90,200,Thunderstorm,thunderstorm with light rain,11d
372924,2020-07-27 20:00:00,Montreal,45.501689,-73.567256,300.47,304.83,298.710,303.150,981,NaN,NaN,94,4.1,270,1.85,NaN,NaN,NaN,90,701,Mist,mist,50d
372999,2020-07-30 23:00:00,Montreal,45.501689,-73.567256,297.59,295.04,292.590,300.370,990,NaN,NaN,54,5.7,260,NaN,NaN,NaN,NaN,75,211,Thunderstorm,thunderstorm,11d


We see that we indeed have duplicates, i-e sometimes there are two rows of information for a given hour. The only difference comes in the 'weather_id', 'weather_main', 'weather_description' and 'weather_icon'. These are categorical features, so we cannot average them. We decide to keep one of the duplicates at random, as otherwise we will have trouble when resambling datas per row. 

In [7]:
df_data = df_data.drop_duplicates(subset='dt_iso', keep="first")

In [8]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 364512 entries, 0 to 373024
Data columns (total 23 columns):
dt_iso                 364512 non-null datetime64[ns]
city_name              364512 non-null object
lat                    364512 non-null float64
lon                    364512 non-null float64
temp                   364512 non-null float64
feels_like             364512 non-null float64
temp_min               364512 non-null float64
temp_max               364512 non-null float64
pressure               364512 non-null int64
sea_level              0 non-null float64
grnd_level             0 non-null float64
humidity               364512 non-null int64
wind_speed             364512 non-null float64
wind_deg               364512 non-null int64
rain_1h                29698 non-null float64
rain_3h                1594 non-null float64
snow_1h                11440 non-null float64
snow_3h                911 non-null float64
clouds_all             364512 non-null int64
weather_id     

We set 'dt_iso' as index. 

In [9]:
df_data = df_data.set_index('dt_iso')

We see that there is no value for 'sea_level' and 'grnd_level' so we drop those columns. Moreover, 'city_name', 'lat', 'lon' are irrelevant as they never change. Also, the 'weather_description' and 'weather_icon' contains the same information as the 'weather_id', so we only keep weather_id. The 'weather_main' also does not contain more information than the 'weather_id' but we keep it for now, as they are nice divisions of features, compared to 'weather_id' which could contain too many categorical features. Finally, the 'temp_min' and 'temp_max' features are deviations from 'temp', which we will not use for our models.   

In [10]:
df_data = df_data.drop(columns = ['sea_level', 'grnd_level', 'city_name', 'lat', 'lon', 'weather_description', 'weather_icon', 'temp_min', 'temp_max'])
print(df_data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 364512 entries, 1979-01-01 00:00:00 to 2020-07-31 23:00:00
Data columns (total 13 columns):
temp            364512 non-null float64
feels_like      364512 non-null float64
pressure        364512 non-null int64
humidity        364512 non-null int64
wind_speed      364512 non-null float64
wind_deg        364512 non-null int64
rain_1h         29698 non-null float64
rain_3h         1594 non-null float64
snow_1h         11440 non-null float64
snow_3h         911 non-null float64
clouds_all      364512 non-null int64
weather_id      364512 non-null int64
weather_main    364512 non-null object
dtypes: float64(7), int64(5), object(1)
memory usage: 38.9+ MB
None


In [11]:
df_data.head()

,temp,feels_like,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main
dt_iso,,,,,,,,,,,,,
1979-01-01 00:00:00,275.12,269.76,1025,80,4.6,140,NaN,NaN,NaN,NaN,90,300,Drizzle
1979-01-01 01:00:00,275.08,271.53,1023,80,2.0,90,NaN,NaN,NaN,NaN,90,600,Snow
1979-01-01 02:00:00,275.06,271.16,1022,80,2.5,120,NaN,NaN,NaN,NaN,90,804,Clouds
1979-01-01 03:00:00,275.97,267.30,1021,86,9.7,160,NaN,NaN,NaN,NaN,90,804,Clouds
1979-01-01 04:00:00,276.32,267.88,1019,93,9.7,160,0.5,NaN,NaN,NaN,90,500,Rain


We get the possible values for 'weather_main' and 'weather_id'.

In [12]:
print('The possible values for "weather_main" are:' ,df_data.weather_main.unique(), '\n')
print('The possible values for "weather_id" are:', df_data.weather_id.unique())

The possible values for "weather_main" are: ['Drizzle' 'Snow' 'Clouds' 'Rain' 'Fog' 'Clear' 'Haze' 'Mist'
 'Thunderstorm' 'Smoke'] 

The possible values for "weather_id" are: [300 600 804 500 501 741 520 801 803 800 620 721 601 802 602 701 521 201
 211 502 301 621 711 612 511 522 321 202 302 503 200]


We see that there are 11 categories in 'weather_main' and 32 categories in 'weather_id'.

We put these datas into a csv file, which we will clean for different models. 

In [13]:
df_data.to_csv("weather_data_initial_clean.csv", header=True, index=True)